In [56]:

### File Address: https://www.sample-videos.com/
%load_ext autoreload
%autoreload 2
import json
import time
import os
from blake3 import blake3
import warnings
import subprocess
warnings.filterwarnings("ignore")
video_path = "demo-3s-small.mp4"
input_height = 240
input_width = 320
output_height = 120
output_width = 160
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
snark_path = "/Users/jojo/Documents/__research/v5/fibonacci/video-resizing-freivalds"
target_pk_file = os.path.join(base_dir, "demo", "test_pk.bin")
target_vk_file = os.path.join(base_dir, "demo", "test_vk.bin")
target_aggregation_pk_file = os.path.join(base_dir, "demo", "test_aggregation_pk.bin")
target_aggregation_vk_file = os.path.join(base_dir, "demo", "test_aggregation_vk.bin")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
!export PATH=$PATH:/usr/local/go/bin && go version

go version go1.24.0 darwin/arm64


# Init ELF

In [58]:
start_time = time.time()
with open(os.devnull, 'w') as devnull:
    subprocess.run(f"cd {snark_path}/program && cargo prove build", 
                shell=True, stdout=devnull, stderr=devnull)
with open(os.devnull, 'w') as devnull:
    subprocess.run(f"cd {snark_path}/aggregation && cargo prove build", 
                shell=True, stdout=devnull, stderr=devnull)
end_time = time.time()
print(f"Time taken to build ELF: {end_time - start_time} seconds")

Time taken to build ELF: 1.697045087814331 seconds


# Verifier Setup Pk and Vk

In [59]:
start_time = time.time()
config = {
    "target_pk_file": target_pk_file,
    "target_vk_file": target_vk_file,
    "target_aggregation_pk_file": target_aggregation_pk_file,
    "target_aggregation_vk_file": target_aggregation_vk_file
}
with open(os.path.join(snark_path, "init", "config.json"), "w") as f:
    json.dump(config, f)
with open(os.devnull, 'w') as devnull:
    subprocess.run(f"cd {snark_path}/init && RUST_LOG=info cargo run --release -- --prove", 
                shell=True, stdout=devnull, stderr=devnull)
end_time = time.time()
print(f"Time taken to generate Pk and Vk: {end_time - start_time} seconds")

Time taken to generate Pk and Vk: 5.255376100540161 seconds


# Prover

### Decompose images from the video

In [60]:
#show the video of file video_path
from IPython.display import display, Video
display(Video(video_path, embed=True))


In [61]:
# Use ffmpeg to decompose
import os
# Create directories if they don't exist
if not os.path.exists("frames"):
    os.makedirs("frames")
if not os.path.exists("decomposed_frames"):
    os.makedirs("decomposed_frames")

os.system(f"ffmpeg -i {video_path} frames/output_%04d.png")

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

0

In [62]:
import sys
sys.path.append(os.path.abspath("../"))
from scripts.decompose_image import decompose_image 
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("frames", i), os.path.join("decomposed_frames", i[:-4]))

In [63]:
### Generate the targeted File 
## Shared knowledge between prover and verifier
if os.path.exists("resized.mp4"):
    os.remove("resized.mp4")
os.system(f"ffmpeg -i {video_path} -vf scale=160:120 resized.mp4")
if not os.path.exists("resized_frames"):
    os.makedirs("resized_frames")
if not os.path.exists("decomposed_resized_frames"):
    os.makedirs("decomposed_resized_frames")

# Check if resized.mp4 exists and ask for overwrite permission

os.system(f"ffmpeg -i resized.mp4 resized_frames/output_%04d.png")
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("resized_frames", i), os.path.join("decomposed_resized_frames", i[:-4]))

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

In [64]:
display(Video("resized.mp4", embed=True))

In [ ]:
# Generate Proof from rust 
if not os.path.exists("proof"):
    os.makedirs("proof")
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
test_counts = 10
for i in sorted(os.listdir("decomposed_frames")):
    start_time = time.time()
    print("generating proof for ", i)
    if i.endswith(".txt"):
        config = {
            "input_width": input_width,
            "input_height": input_height,
            "output_width": output_width,
            "output_height": output_height,
            "input_file": os.path.join(base_dir, "demo/decomposed_frames", i),
            "target_file": os.path.join(base_dir, "demo/decomposed_resized_frames", i),
            "target_prove_file": os.path.join(base_dir, "demo/proof", i[:-4] + "_proof-with-pis.bin"),
            "target_pk_file": target_pk_file
        }
        with open(os.path.join(snark_path, "script", "config.json"), "w") as f:
            json.dump(config, f)
        with open(os.devnull, 'w') as devnull:
            subprocess.run(f"cd {snark_path}/script && RUST_LOG=info cargo run --release -- --prove", 
                        shell=True, stdout=devnull, stderr=devnull)
       # os.system(f"cd {snark_path}/script && RUST_LOG=info cargo run --release -- --prove")
    end_time = time.time()
    print(f"Time taken to generate proof for {i}: {end_time - start_time} seconds")
    test_counts -= 1
    if test_counts == 0:
        break    


generating proof for  output_0001_B.txt


# Using Proof Network

In [32]:
# Generate Proof from rust 
if not os.path.exists("proof"):
    os.makedirs("proof")
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
test_counts = 10
for i in sorted(os.listdir("decomposed_frames")):
    start_time = time.time()
    print("generating proof for ", i)
    if i.endswith(".txt"):
        config = {
            "input_width": input_width,
            "input_height": input_height,
            "output_width": output_width,
            "output_height": output_height,
            "input_file": os.path.join(base_dir, "demo/decomposed_frames", i),
            "target_file": os.path.join(base_dir, "demo/decomposed_resized_frames", i),
            "target_prove_file": os.path.join(base_dir, "demo/proof", i[:-4] + "_proof-with-pis.bin"),
            "target_pk_file": target_pk_file
        }
        with open(os.path.join(snark_path, "script", "config.json"), "w") as f:
            json.dump(config, f)
        with open(os.devnull, 'w') as devnull:
            subprocess.run(f"cd {snark_path}/script && SP1_PRIVATE_KEY=0x802a3b2e614919ee3843a7747b0faaaca7118ad6df44927d82f726d9be98e484 RUST_LOG=info cargo run --release -- --prove", 
                        shell=True, stdout=devnull, stderr=devnull)
       # os.system(f"cd {snark_path}/script && RUST_LOG=info cargo run --release -- --prove")
    end_time = time.time()
    print(f"Time taken to generate proof for {i}: {end_time - start_time} seconds")
    test_counts -= 1
    if test_counts == 0:
        break    


generating proof for  output_0001_B.txt


KeyboardInterrupt: 

# Aggregation Proof

In [ ]:
start_time = time.time()
config = {
    "vk" :  target_vk_file,
    "aggregation_pk": target_aggregation_pk_file,
    "pk" : target_pk_file,
    "target_prove_file": "final_proof.bin",
    "proof_files":[]
}
for i in sorted(os.listdir("proof")):
    config["proof_files"].append( os.path.join(base_dir, "demo","proof", i))
# config["proof_files"] = config["proof_files"][:5]
with open(os.path.join(snark_path, "aggregate_script", "config.json"), "w") as f:
    json.dump(config, f)
with open(os.devnull, 'w') as devnull:
    subprocess.run(f"cd {snark_path}/aggregate_script && RUST_BACKTRACE=1 cargo run --release -- --prove", 
                shell=True, stdout=devnull, stderr=devnull)
# os.system(f"cd {snark_path}/aggregate_script && RUST_BACKTRACE=1 cargo run --release -- --prove")
end_time = time.time()
print("Aggregation Proof Time: ", end_time - start_time)


Aggregation Proof Time:  537.7143270969391


#  Aggregation Verifier

In [28]:
start_time = time.time()
config = {
    "target_prove_file": os.path.join(snark_path, "aggregate_script", "final_proof.bin"),
    "target_vk_file": target_aggregation_vk_file
}
with open(os.path.join(snark_path, "aggregate_verifier", "config.json"), "w") as f:
    json.dump(config, f)
with open(os.devnull, 'w') as devnull:
    subprocess.run(f"cd {snark_path}/aggregate_verifier && cargo run", 
                shell=True, stdout=devnull, stderr=devnull)
# os.system(f"cd {snark_path}/aggregate_verifier && cargo run")
end_time = time.time()
print(end_time - start_time)

17.262555837631226


# Verifier

In [29]:
if not os.path.exists("resized_frames"):
    os.makedirs("resized_frames")
if not os.path.exists("decomposed_resized_frames"):
    os.makedirs("decomposed_resized_frames")

# Check if resized.mp4 exists and ask for overwrite permission

os.system(f"ffmpeg -i resized.mp4 resized_frames/output_%04d.png")
for i in sorted(os.listdir("frames")):
    if i.endswith(".png"):
        r,g,b = decompose_image(os.path.join("resized_frames", i), os.path.join("decomposed_resized_frames", i[:-4]))

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

In [30]:
test_counts = 10
for i in sorted(os.listdir("decomposed_resized_frames")):
    print("verifying proof for ", i)
    if i.endswith(".txt"):
        start_time = time.time()
        prove_file = os.path.join(base_dir, "demo/proof", i[:-4] + "_proof-with-pis.bin")
        config = {
            "target_prove_file": prove_file,
            "target_vk_file": target_vk_file
        }
        with open(os.path.join(snark_path, "verifier", "config.json"), "w") as f:
            json.dump(config, f)
        with open(os.devnull, 'w') as devnull:
            subprocess.run(f"cd {snark_path}/verifier && cargo run", 
                        shell=True, stdout=devnull, stderr=devnull)
        # os.system(f"cd {snark_path}/verifier && cargo run")
        data = []
        with open(os.path.join(base_dir, "demo/decomposed_resized_frames", i), "r") as f:
            for line in f:
                # Split line by spaces and convert each number
                numbers = [int(x) for x in line.strip().split()]
                data.extend(numbers)
        hash_obj = blake3(bytes(data))
        print(hash_obj.hexdigest())
        end_time = time.time()
        print(f"Time taken to verify proof for {i}: {end_time - start_time} seconds")
    test_counts -= 1
    if test_counts == 0:
        break

verifying proof for  output_0001_B.txt
c46149be2e66bc0b4899d725e14d3281df75453e961daf6c64fcf19911864990
Time taken to verify proof for output_0001_B.txt: 17.191483974456787 seconds
verifying proof for  output_0001_G.txt
236150d752ae55abe27b9f8407379836f19e2e02cc9478efad79201ac140ffc5
Time taken to verify proof for output_0001_G.txt: 17.560409784317017 seconds
verifying proof for  output_0001_R.txt
3129a6601880af99970a4ac518d4ed7f2ee9ae609be67f1e6a67a044be0cd358
Time taken to verify proof for output_0001_R.txt: 17.023669004440308 seconds
verifying proof for  output_0002_B.txt
0ccc1c2f0f3c0880a37817223a48f703672ff2e2d9c3ed8c0bac6a35aafa7f1f
Time taken to verify proof for output_0002_B.txt: 17.27041482925415 seconds
verifying proof for  output_0002_G.txt
44f4b483703b5dff7ef279f41d48d36e0600b5c99bdf55d6b4e5d520046aa302
Time taken to verify proof for output_0002_G.txt: 16.972655057907104 seconds
